In [1]:
import pandas as pd
import xgboost as xgb

import feather

from tqdm import tqdm_notebook as tqdm

In [2]:
D = 2 ** 20

def hash_element(el):
    h = hash(el) % D
    if h < 0:
        h = h + D

    return str(h)

In [3]:
val = True

if val:
    df_train = feather.read_dataframe('feather/df_train.feather')
    df_test = feather.read_dataframe('feather/df_val.feather')
    out = 'tmp'
else:
    df_train = feather.read_dataframe('feather/df_train_all.feather')
    df_test = feather.read_dataframe('feather/df_test.feather')
    out = 'tmp_test'

In [4]:
field_mapping = {'turn': 3, 'player': 2, 'opponent': 1}

def ffm_row(d):
    decision = d.get('decision', 0)

    result = []

    for k in sorted(d.keys()):
        if k == 'decision':
            continue

        if k == 'turn':
            result.append('3:%s:1' % d['turn'])
            continue

        f, c = k.split('.', maxsplit=1)

        feature = hash_element('%s=%s' % (c, d[k]))
        field = field_mapping[f]

        result.append('%s:%s:1' % (field, feature))

    return '%s %s' % (decision, ' '.join(result))

In [5]:
for _, row in tqdm(df_train.iloc[:100].iterrows()):
    res = ffm_row(row.to_dict())
    print(res[:10])

0 1:546386
0 1:546386
1 1:546386
0 1:546386
0 1:546386
1 1:546386
1 1:102869
0 1:546386
1 1:546386
1 1:102869
1 1:546386
0 1:546386
1 1:546386
0 1:546386
0 1:546386
0 1:546386
1 1:546386
0 1:546386
0 1:546386
1 1:546386
1 1:546386
0 1:546386
1 1:546386
1 1:546386
1 1:546386
1 1:546386
0 1:546386
1 1:546386
1 1:546386
0 1:546386
1 1:546386
0 1:546386
0 1:546386
1 1:935061
1 1:546386
1 1:546386
1 1:546386
0 1:102869
1 1:546386
0 1:546386
0 1:546386
1 1:546386
1 1:546386
0 1:546386
1 1:546386
1 1:546386
0 1:546386
1 1:546386
0 1:50886:
1 1:546386
0 1:546386
0 1:546386
0 1:546386
1 1:546386
1 1:546386
0 1:546386
0 1:546386
1 1:546386
1 1:546386
1 1:546386
0 1:546386
1 1:985750
1 1:546386
1 1:546386
0 1:546386
0 1:546386
1 1:546386
0 1:546386
0 1:546386
1 1:546386
0 1:546386
0 1:546386
1 1:546386
0 1:546386
0 1:985750
0 1:546386
1 1:546386
0 1:546386
1 1:985750
1 1:546386
1 1:546386
1 1:546386
0 1:546386
0 1:546386
0 1:546386
1 1:546386
0 1:546386
0 1:546386
0 1:546386
0 1:546386
0 1:546386

In [6]:
with open('ffm/%s_train_ffm.txt' % (out), 'w') as f:

    for _, row in tqdm(df_train.iterrows()):
        res = ffm_row(row.to_dict())
        f.write(res + '\n')

In [7]:
with open('ffm/%s_test_ffm.txt' % (out), 'w') as f:

    for _, row in tqdm(df_test.iterrows()):
        res = ffm_row(row.to_dict())
        f.write(res + '\n')

In [8]:
val = False

if val:
    df_train = feather.read_dataframe('feather/df_train.feather')
    df_test = feather.read_dataframe('feather/df_val.feather')
    out = 'tmp'
else:
    df_train = feather.read_dataframe('feather/df_train_all.feather')
    df_test = feather.read_dataframe('feather/df_test.feather')
    out = 'tmp_test'

In [9]:
with open('ffm/%s_train_ffm.txt' % (out), 'w') as f:

    for _, row in tqdm(df_train.iterrows()):
        res = ffm_row(row.to_dict())
        f.write(res + '\n')

In [10]:
with open('ffm/%s_test_ffm.txt' % (out), 'w') as f:

    for _, row in tqdm(df_test.iterrows()):
        res = ffm_row(row.to_dict())
        f.write(res + '\n')

    ~/soft/libffm/ffm-train -s 8 -k 8 -l 0.000001 -t 15 tmp_test_train_ffm.txt ffm.model
    ~/soft/libffm/ffm-predict tmp_test_test_ffm.txt ffm.model test_pred.txt